In [1]:
import pickle
import pandas as pd
from sklearn.manifold import TSNE

import plotly.express as px

In [2]:
lyrics_df = pd.read_csv("../data/processed/lyrics.csv", sep=";")

In [3]:
with open("../data/processed/song_encodings.pkl", "rb") as infile:
    song_encodings_map = pickle.load(infile)

---

In [28]:
tsne = TSNE(perplexity=10)

In [29]:
song_embeddings = tsne.fit_transform(list(song_encodings_map.values()))

In [31]:
song_df = pd.DataFrame(
    zip(list(song_encodings_map.keys()), song_embeddings[:,0], song_embeddings[:,1]),
    columns=["song_idx", "x", "y"]
)

In [32]:
song_df = song_df.merge(lyrics_df, left_on="song_idx", right_on="song_idx")
song_df.head()

,song_idx,x,y,sentence,artista,musica,letra,lang
0,0,-20.249626,-16.845879,[DaBaby:],Dua Lipa,Levitating (feat. DaBaby),"[DaBaby:]\nBillboard Baby, Dua Lipa make 'em d...",2
1,0,-20.249626,-16.845879,"Billboard Baby, Dua Lipa make 'em dance when i...",Dua Lipa,Levitating (feat. DaBaby),"[DaBaby:]\nBillboard Baby, Dua Lipa make 'em d...",2
2,0,-20.249626,-16.845879,Everybody lookin' for a dance floor to run on,Dua Lipa,Levitating (feat. DaBaby),"[DaBaby:]\nBillboard Baby, Dua Lipa make 'em d...",2
3,0,-20.249626,-16.845879,[Dua Lipa:],Dua Lipa,Levitating (feat. DaBaby),"[DaBaby:]\nBillboard Baby, Dua Lipa make 'em d...",2
4,0,-20.249626,-16.845879,"If you wanna run away with me, I know a galaxy",Dua Lipa,Levitating (feat. DaBaby),"[DaBaby:]\nBillboard Baby, Dua Lipa make 'em d...",2


In [17]:
# song_df.to_csv("../data/processed/songs_with_coordinates.csv", sep=";", index=False)

---

In [37]:
unique_df = song_df[["song_idx", "x", "y", "artista", "musica", "letra"]].drop_duplicates()

In [38]:
unique_df.head()

,song_idx,x,y,artista,musica,letra
0,0,-20.249626,-16.845879,Dua Lipa,Levitating (feat. DaBaby),"[DaBaby:]\nBillboard Baby, Dua Lipa make 'em d..."
97,1,-23.208698,-15.525610,Dua Lipa,Don't Start Now,"If you don't wanna see me\n\nDid a full 180, c..."
147,2,27.075216,38.009338,Dua Lipa,Break My Heart,I've always been the one to say the first good...
214,3,-20.640112,-16.691019,Dua Lipa,Levitating,If you wanna run away with me\nI know a galaxy...
297,4,-5.635098,-3.148146,Dua Lipa,New Rules,Talkin' in my sleep at night\nMakin' myself cr...


In [34]:
fig = px.scatter(
    unique_df.sort_values("artista"),
    x="x", y="y",
    color="artista",
    hover_data=["musica"],
    opacity=0.5
)
fig.show()